# Data Frame


## Creacion de un dataframe a partir de un fichero csv

{"nombre":"Eleven", "edad": 12, "sexo":"M"}

{"nombre":"Hopper", "edad": 45, "sexo":"H"}

{"nombre":"Will", "edad": 11, "sexo":"H"}

{"nombre":"Max", "edad": 11, "sexo":"M"}

{"nombre":"Mike", "edad": 11, "sexo":"H"}

{"nombre":"Lucas", "edad": 11, "sexo":"H"}

{"nombre":"Dustin", "edad": 11, "sexo":"H"}

{"nombre":"Eleven", "edad": 12, "sexo":"M"}

{"nombre":"Nancy", "edad": 23, "sexo":"M"}

{"nombre":"Jonathan", "edad": 24, "sexo":"H"}


In [1]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark3").getOrCreate()
print(spark)

df = spark.read.options(header='True', inferSchema='True').json("stranger.json")
##df = spark.read.json("stranger.json")






In [2]:
#Mostrar numero de elementos de un Dataframe
df.count()

10

In [3]:
#mostramos las columnas de un dataframe
df.columns

['edad', 'nombre', 'sexo']

In [4]:
#mostramos la estructura de un dataframe
df.printSchema()

root
 |-- edad: long (nullable = true)
 |-- nombre: string (nullable = true)
 |-- sexo: string (nullable = true)



In [5]:
#ver el contenido de un dataframe a partir del comando show

df.show()

+----+--------+----+
|edad|  nombre|sexo|
+----+--------+----+
|  12|  Eleven|   M|
|  45|  Hopper|   H|
|  11|    Will|   H|
|  11|     Max|   M|
|  11|    Mike|   H|
|  11|   Lucas|   H|
|  11|  Dustin|   H|
|  12|  Eleven|   M|
|  23|   Nancy|   M|
|  24|Jonathan|   H|
+----+--------+----+



In [6]:
#seleccionamos un subconjunto de las columna
dfNombreSexo = df.select("nombre", "sexo")
dfNombreSexo.show()




+--------+----+
|  nombre|sexo|
+--------+----+
|  Eleven|   M|
|  Hopper|   H|
|    Will|   H|
|     Max|   M|
|    Mike|   H|
|   Lucas|   H|
|  Dustin|   H|
|  Eleven|   M|
|   Nancy|   M|
|Jonathan|   H|
+--------+----+



In [7]:
##filtramos los datos, y que nos muestre solo las personas mayores de edad

dfMayores = df.filter(df.edad>=18)
dfMayores. show()

+----+--------+----+
|edad|  nombre|sexo|
+----+--------+----+
|  45|  Hopper|   H|
|  23|   Nancy|   M|
|  24|Jonathan|   H|
+----+--------+----+



In [8]:
#primer elementos
df.first()


Row(edad=12, nombre='Eleven', sexo='M')

In [9]:
#head -->  es quivalente al take - recupera los primeros elementos
df.head(2)

[Row(edad=12, nombre='Eleven', sexo='M'),
 Row(edad=45, nombre='Hopper', sexo='H')]

In [10]:
#groupby
#agrupa los elementos en base a los valores de una determinada columna
# en este caso agrupa las personas que tengan la misma edad y nos devuelve el numero de repeticiones

df.groupBy("edad").count().show()

+----+-----+
|edad|count|
+----+-----+
|  12|    2|
|  11|    5|
|  23|    1|
|  45|    1|
|  24|    1|
+----+-----+



In [11]:
#describe nos devuelve un resumen estadístico de los datos
df.describe().show()

+-------+------------------+------+----+
|summary|              edad|nombre|sexo|
+-------+------------------+------+----+
|  count|                10|    10|  10|
|   mean|              17.1|  null|null|
| stddev|11.049886877249017|  null|null|
|    min|                11|Dustin|   H|
|    max|                45|  Will|   M|
+-------+------------------+------+----+



# Ahora vamos a crear un data frame desde un fichero .txt


In [12]:
dftxt = spark.read.text("stranger.txt")
dftxt.show()

+---------------+
|          value|
+---------------+
|  Eleven, 12, M|
|  Hopper, 45, H|
|    Will, 11, H|
|     Max, 11, M|
|    Mike, 11, H|
|   Lucas, 11, H|
|  Dustin, 11, H|
|  Eleven, 12, M|
|   Nancy, 23, M|
|Jonathan, 24, H|
+---------------+



In [13]:
dfcol1= dftxt.selectExpr("split(value, ', ') as col1").show()


+-----------------+
|             col1|
+-----------------+
|  [Eleven, 12, M]|
|  [Hopper, 45, H]|
|    [Will, 11, H]|
|     [Max, 11, M]|
|    [Mike, 11, H]|
|   [Lucas, 11, H]|
|  [Dustin, 11, H]|
|  [Eleven, 12, M]|
|   [Nancy, 23, M]|
|[Jonathan, 24, H]|
+-----------------+



In [15]:
#import spark.implicits

def partir(fila):
    elements = fila.split(", ")
    return elements                                   
 
#print(partir("hola , fff ,ffff"))            
          
    
rdd1 = dftxt.rdd  
#rdd1.collect()
rddfile = rdd1.map(lambda x: partir(x[0]))
#rddfile.collect()
rddfile.toDF(schema=['a', 'b', 'c']).show()

+--------+---+---+
|       a|  b|  c|
+--------+---+---+
|  Eleven| 12|  M|
|  Hopper| 45|  H|
|    Will| 11|  H|
|     Max| 11|  M|
|    Mike| 11|  H|
|   Lucas| 11|  H|
|  Dustin| 11|  H|
|  Eleven| 12|  M|
|   Nancy| 23|  M|
|Jonathan| 24|  H|
+--------+---+---+

